In [2]:
#!/usr/bin/env python3
import matplotlib.pyplot as plt
from __future__ import print_function
import sys
import tarfile
import numpy
import itertools
from sklearn import *
import scipy
from collections import OrderedDict
import matplotlib.cm as cm

(COMPUTING, COMBINE, CALL_INDIRECT, CALL_REC, CALL, FUNCTION, COMPOUND, BRANCH, LOOP) = (i for i in range(-9, 0, 1))

topk = OrderedDict()

rootResult = list()
uniqId = 0
compNum = 0
commNum = 0


nprocs = [16]

class Node(object):
    def __init__(self,
                 i,
                 id,
                 type,
                 #firstType,
                 #lastType,
                 numChildren,
                 dirID,
                 fileID,
                 lineNum,
                 exitLineNum,
                 sampleCount,
                 sumTime):
        self.uniqueId = i
        self.id = id
        self.type = type
        #self.firstType = firstType
        #self.lastType = lastType
        self.children = [None] * numChildren
        self.dirID = dirID
        self.fileID = fileID
        self.lineNum = lineNum
        self.exitLineNum = exitLineNum
        self.sumTime = sumTime
        self.sampleCount = sampleCount
        if numChildren == 0 :
            self.flagNode =  1 # init flagNode =1 to all leaves
        else :
            self.flagNode =  0
        # sumTime only
        self.sumTimeList = [sumTime]
        self.sumTimeSecList = []
        self.hasMulTotalTime = 0
        self.pred = 0
        self.crossPreds = []
        
        #print (  i,id,type,numChildren,dirID,fileID,lineNum,sumTime)

    def gatherOne(self, node):
        self.sumTimeSecList.append(node.sumTimeList)
        for i in range(len(self.children)):
            self.children[i].gatherOne(node.children[i])
    
    def gatherStat(self, node):
        self.sumTimeList += node.sumTimeList
        for i in range(len(self.children)):
            self.children[i].gatherStat(node.children[i])

    def getNodeNum(self):
        num = 1
        if self.type < 0 and self.type not in (FUNCTION, COMPOUND, BRANCH, LOOP):
            num = 0
        if len(self.children) == 0:
            return num
        for child in self.children:
            num += child.getNodeNum()
        return num

    

    

def readTree(f):
    global uniqId
    types = [int, int, int, int, int, int, int, int, float]
    args = list(map(lambda x: x[0](x[1]), zip(types, f.readline().decode('utf-8').strip().split(' '))))
    args.insert(0, uniqId)
    uniqId += 1
    node = Node(*args)
    for i in range(len(node.children)):
        node.children[i] = readTree(f)
    return node

def gatherInfo(node):
    global commNum,compNum
    if node.type == COMPUTING:
        compNum += 1
    elif node.type >= 0:
        commNum += 1
    for child in node.children:
        gatherInfo(child)

def trimTree(node):
    for child in node.children:
        trimTree(child)

    remove = [False] * len(node.children)
    for i in range(len(node.children)):
        child = node.children[i]
        if child.type in (CALL_REC, CALL_INDIRECT):
            assert len(child.children) == 0
            remove[i] = True
        elif child.type == CALL:
            assert len(child.children) == 1 and child.children[0].type == FUNCTION
            node.children[i] = child.children[0]
        elif child.type == BRANCH:
            assert child.sumTime == 0
            sumTime = 0
            for cc in child.children:
                sumTime += cc.sumTime
            child.sumTime = sumTime
            child.sumTimeList = [sumTime]
    newChildren = []
    for i in range(len(node.children)):
        if not remove[i]:
            newChildren.append(node.children[i])
    node.children = newChildren

def printTime(node, depth):
    print(' ' * depth, end='')
    print([node.type, node.sumTime, node.numEntries])
    for child in node.children:
        printTime(child, depth + 1)


def gatherTime(node):
    def gatherTimeRec(node, out):
        out.append((node.type, node.sumTime))
        for child in node.children:
            gatherTimeRec(child, out)
    ret = []
    gatherTimeRec(node, ret)
    return ret


def sumLeaves(node):
    if len(node.children) == 0:
        return node.sumTime
    sumTime = 0
    for child in node.children:
        sumTime += sumLeaves(child)
    return sumTime

def mulTotalTime(node):
    if node.hasMulTotalTime == 0:
        for i in range(len(node.sumTimeSecList)):
            for j in range(len(node.sumTimeSecList[i])):
                node.sumTimeSecList[i][j] = node.sumTimeSecList[i][j] * totalTime[i]
        node.hasMulTotalTime = 1
    for i in range(len(node.children)):
        mulTotalTime( node.children[i])

if __name__ == '__main__':
    # overhead()
    # exit(0)
    #bms = ['bt', 'cg', 'ep', 'ft', 'mg', 'sp', 'lu', 'is']
    #bms = [ 'cg' , 'ep' , 'ft' , 'mg' ,'lu' , 'is']

    #bms = ['cg']
    #bms = ['sweep3D']
    #bms = ['Lulesh2.0']
    #bms = ['amg']
    bms = ['chol_decom_MPI']
    #np1 = [16, 32, 64, 128, 256, 512, 1024, 2048]
    #np1 = [16, 32, 64, 128, 256] # , 512, 1024, 2048]
    # np1 = [16, 32, 64, 128, 256, 512]
    #np2 = [4, 9, 16, 36, 64] #16, 36, 64, 121, 256, 529, 1024]
    #np2 = [16, 36, 64, 121, 256, 2025] #, 1024]
    #tf = tarfile.open('./TEST_ft_7.tar.gz', 'r')
    #tf = tarfile.open('./npb_ft_D_16-512_TH20190314_64-4node.tar.gz', 'r')
    #tf = tarfile.open('./npb_ft_D_16-512_TH20190314_64-4node.tar.gz', 'r')
    #tf = tarfile.open('./npb_D_16-256_TH20190305.tar.gz','r')
    #tf = tarfile.open('./GRAPH500.tar.gz', 'r')
    #tf = tarfile.open('./Lulesh2.0.tar.gz','r')
    #tf = tarfile.open('./SWEEP3D.tar.gz','r')
    #tf = tarfile.open('./AMG.tar.gz','r')
    #tf = tarfile.open('./choldecomMPI.tar.gz','r')
    tf = tarfile.open('./choldecomMPItest.tar.gz','r')
    #tf = tarfile.open('./npb_info_collect_org_8-9.tar.gz', 'r')
    
    for bm in bms:
        
        nps = nprocs
        # print(bm)

        mergedTrees = []
        for np in nps: 
            # Read trees
            uniqId = 0
            trees = []
            # for rank in range(np):
            for rank in range(1):  # read only process 1 : stat0.txt
                #path = './npbrun/C/%s/%d/stat%d.txt' % (bm, np, rank)
                path = '%s/%d/stat%d.txt' % (bm, np, rank)
                #path = './run-with-all-leafnode/A/%s/%d/stat%d.txt' % (bm, np, rank)
                f = tf.extractfile(tf.getmember(path))
                root = readTree(f)
                # print(root.getNodeNum())
                #trimTree(root)
                trees.append(root)

            # mergeTrees(trees)
            mergedTrees.append(trees[0]) # contains stat0.txt of all nps

        for i in range(1, len(mergedTrees)):
            mergedTrees[0].gatherOne(mergedTrees[i])

        gatheredTree = mergedTrees[0]  # merge[i] is merged to merge[0] 
        print ("----------------------",bm,"----------------------")
        print ("-----------------------------------------------")
        print(uniqId)
        
        gatherInfo(gatheredTree)
        print("comp",compNum)
        print("comm",commNum)
        compNum = 0
        commNum = 0

---------------------- chol_decom_MPI ----------------------
-----------------------------------------------
70
comp 37
comm 5
